In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
data_folder = '/content/drive/MyDrive/asl_alphabet_train'

In [4]:
train_transforms = transforms.Compose([
    
    transforms.RandomHorizontalFlip(),
    
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [5]:
ls

drive/  sample_data/


In [6]:
cd drive

/content/drive


In [7]:
cd MyDrive

/content/drive/MyDrive


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.optim import Adam

In [9]:
d=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
num_classes=3
from torchvision.models import mobilenet_v2
model=models.mobilenet_v2(pretrained=False)
features = model.last_channel  # Access last_channel attribute
model.classifier = nn.Linear(features, num_classes)  # Replace the final classification layer


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [11]:
#for param in model.parameters():
#    param.requires_grad = False
#model.classifier[-1] = nn.Linear(model.last_channel, num_classes)
model = model.to(d)

In [12]:
d

device(type='cuda')

In [13]:
train_transforms = transforms.Compose([
    
    transforms.RandomHorizontalFlip(),
    
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [14]:
optimizer=Adam(model.parameters(),lr=0.001)
criterion=nn.CrossEntropyLoss()

In [15]:
dataset = datasets.ImageFolder(r"asl_alphabet_train", transform=test_transforms)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_dataset.transform=train_transforms
from torch.utils.data import DataLoader
loader_train=DataLoader(train_dataset,batch_size=32,shuffle=True )
loader_test=DataLoader(test_dataset,batch_size=32,shuffle=True)

In [ ]:
num_epochs = 10
total_steps = len(loader_train)

for epoch in range(num_epochs):
    for i, data in enumerate(loader_train):
        images, labels = data[0].to(d), data[1].to(d)
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backprop and optimisation
        
        loss.backward()
        optimizer.step()
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))

Epoch [1/10], Step [100/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [1/10], Step [200/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [2/10], Step [100/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [2/10], Step [200/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [3/10], Step [100/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [3/10], Step [200/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [4/10], Step [100/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [4/10], Step [200/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [5/10], Step [100/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [5/10], Step [200/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [6/10], Step [100/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [6/10], Step [200/200], Loss: 0.0000, Accuracy: 100.00%
Epoch [7/10], Step [100/200], Loss: 0.0000, Accuracy: 100.00%


In [ ]:
model.eval()
with torch.no_grad():
  correct = 0
  total=0
  for image,label in loader_test:
    images,label=images.to(d),labels.to(d)
    result=model(images)
    _,predicted=torch.max(result.data,1)
    total=total+labels.size(0)
    correct=correct+(predicted==labels).sum().items()
    print(f'Accuracy {100 * correct / total:.2f}%')